# Tests

In [2]:
import os
import sys
import logging
import requests
import json
import re
import pandas as pd

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from lib.nanuq import Nanuq
from lib.gapp import Phenotips
from lib.gapp import BSSH
from lib.samplesheet import SampleSheet

nq    = Nanuq()
pho   = Phenotips()
bssh  = BSSH()

class Args:
    # Simulate an argparse instance
    #
    def __init__(self, run='LH00336_0006', level='info'):
        self.run   = run
        self.level = level

args = Args(level='debug')
vars(args)

{'run': 'LH00336_0006', 'level': 'debug'}

In [3]:
sheet = SampleSheet('D:\\HSJ\\Projects\\PRAGMatIQ\\Runs\\Seq_10B_PRAG_20240130.csv')
sheet.sections['DragenGermline_Data'][0]

['Sample_ID',
 'ReferenceGenomeDir',
 'VariantCallingMode',
 'QcCoverage1BedFile',
 'QcCoverage2BedFile',
 'QcCoverage3BedFile',
 'QcCrossContaminationVcfFile']

In [13]:
with open('D:\\HSJ\\Projects\\PRAGMatIQ\\Runs\\Seq_10B_PRAG_20240130.csv', 'r') as fh:
    content = fh.read().splitlines()
content

['[Header],',
 'FileFormatVersion,2',
 'RunName,Seq_10B_PRAG_20240130',
 'InstrumentPlatform,NovaSeqXSeries',
 'IndexOrientation,Forward',
 '',
 '[Reads]',
 'Read1Cycles,151',
 'Read2Cycles,151',
 'Index1Cycles,10',
 'Index2Cycles,10',
 '',
 '[Sequencing_Settings]',
 'LibraryPrepKits,IlluminaDNAPCRFree',
 '',
 '[BCLConvert_Settings]',
 'SoftwareVersion,4.1.23',
 'AdapterRead1,CTGTCTCTTATACACATCT+ATGTGTATAAGAGACA',
 'AdapterRead2,CTGTCTCTTATACACATCT+ATGTGTATAAGAGACA',
 'OverrideCycles,Y151;I10;I10;Y151',
 'FastqCompressionFormat,gzip',
 '',
 '[BCLConvert_Data]',
 'Lane,Sample_ID,Index,Index2',
 '1,24159,TCTCATGATA,AACACGTGGA',
 '2,24159,TCTCATGATA,AACACGTGGA',
 '3,24159,TCTCATGATA,AACACGTGGA',
 '4,24159,TCTCATGATA,AACACGTGGA',
 '5,24159,TCTCATGATA,AACACGTGGA',
 '6,24159,TCTCATGATA,AACACGTGGA',
 '7,24159,TCTCATGATA,AACACGTGGA',
 '8,24159,TCTCATGATA,AACACGTGGA',
 '1,24160,CGAGGCCAAG,GTGTTACCGG',
 '2,24160,CGAGGCCAAG,GTGTTACCGG',
 '3,24160,CGAGGCCAAG,GTGTTACCGG',
 '4,24160,CGAGGCCAAG,GTGTT

In [15]:
content = nq.get_samplesheet('LH00336_0009').text.splitlines()
content

['[Header]',
 'FileFormatVersion,2',
 'RunName,LH00336_0006',
 'InstrumentPlatform,NovaSeq',
 'IndexOrientation,Forward',
 '',
 '[Reads]',
 'Read1Cycles,151',
 'Read2Cycles,151',
 'Index1Cycles,10',
 'Index2Cycles,10',
 '',
 '[BCLConvert_Settings]',
 'SoftwareVersion,3.8.4',
 'FastqCompressionFormat,gzip',
 '',
 '[BCLConvert_Data]',
 'Lane,Sample_ID,Index,Index2,Sample_Project',
 '1,23399,AGAGGCAACC,CTAATGATGG,PRAGMatIQ_CHUSJ',
 '1,23402,CCATCATTAG,GGTTGCCTCT,PRAGMatIQ_CHUSJ',
 '1,23404,GATAGGCCGA,CGCACATGGC,PRAGMatIQ_CHUSJ',
 '1,23405,ATGGTTGACT,GGCCTGTCCT,PRAGMatIQ_CHUSJ',
 '1,23406,TATTGCGCTC,CTGTGTTAGG,PRAGMatIQ_CHUSJ',
 '1,23408,TTCTACATAC,CTAACTGTAA,PRAGMatIQ_CHUSJ',
 '1,23411,AACCATAGAA,GGCGAGATGG,PRAGMatIQ_CHUSJ',
 '1,23413,GGTTGCGAGG,AATAGAGCAA,PRAGMatIQ_CHUSJ',
 '1,23414,TAAGCATCCA,TCAATCCATT,PRAGMatIQ_CHUSJ',
 '1,23415,ACCACGACAT,TCGTATGCGG,PRAGMatIQ_CHUSJ',
 '1,23420,GGTAACTCGC,AAGTTGGTGA,PRAGMatIQ_CHUQ',
 '1,23421,ACCGGCCGTA,TGGCAATATT,PRAGMatIQ_CHUQ',
 '1,23422,TGTAATCGAC

In [14]:
samples = []
for line in content:
    if line.startswith('['):
        section = line
    else:
        if section.startswith('[DragenGermline_Data]'):
            cols = line.split(',')
            if not line.startswith('Sample_ID') and len(cols) > 1:
                samples.append(cols[0])
samples


['24159',
 '24160',
 '24161',
 '24162',
 '24163',
 '24164',
 '24212',
 '24213',
 '24214',
 '24215',
 '24216',
 '24217',
 '24251',
 '24252',
 '24253',
 '24254',
 '24255',
 '24256']

In [ ]:
# Get authorization token from Emedgene
# Please note - the Authorization header is only valid for a limited time, and
# expires after 8H. In that case, any request made with an expired token will 
# return a 403 code. To resolve, re-do the Login procedure to get a new token.
#
url      = "https://chusaintejustine.emedgene.com/api/auth/api_login/"
payload  = '{"username": "cqgc.bioinfo.hsj@ssss.gouv.qc.ca", "password": "3175CoteSainte-Catherine"}'
headers  = {'Content-Type': 'application/json'}
response = requests.request("POST", url, headers=headers, data=payload)
auth_header = response.json()["Authorization"]
auth_header


In [ ]:
# Get case for EMG634551172 (GM240123)
#
case_id = "EMG634551172"
get_test_response = requests.get(f'https://chusaintejustine.emedgene.com/api/test/{case_id}/', json={}, headers={'Authorization': auth_header})
# with open('D:\\HSJ\\Projects\\PRAGMatIQ\\emg_scripts\\emg_case_example-GM240123.json', 'w') as fh:
#     json.dump(get_test_response.json(), fh, indent=4)
get_test_response.json().keys()


In [ ]:
# Transfer files to aws
# --profile "emedgene" or "emedgene-eval"
aws s3 ls --profile emedgene-eval s3://cac1-prodca-emg-downloads/Ste_Justine_eval/upload